<a href="https://colab.research.google.com/github/Rishika70/Hack_To_Hire/blob/main/Hack_To_Hire.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install transformers[torch] tokenizers datasets evaluate rouge_score sentencepiece huggingface_hub --upgrade

In [3]:
!pip install bleu

  Preparing metadata (setup.py) ... done
  Created wheel for bleu: filename=bleu-0.3-py3-none-any.whl size=5780 sha256=d05d40f5f5ec1cc422687b4c90e94bff2531d4e9b24d37b0fdfb67c1cf6b7edf
  Stored in directory: /root/.cache/pip/wheels/c6/d8/d1/009abe01b8b2c6a14c62d197b510b3cc1076014c22d712c5ce
Successfully built bleu


In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import BertTokenizer, BertModel
from transformers import T5Tokenizer, T5Model
from transformers import GPT2Tokenizer, GPT2Model
from rouge_score import rouge_scorer
from nltk.translate.bleu_score import sentence_bleu
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objs as go
from sklearn.model_selection import train_test_split


# Tokenization

In [7]:
# Tokenize the data
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [4]:
!pip install datasets
#Import the datasets library
from datasets import load_dataset


# Load and split the dataset
dataset = load_dataset("toughdata/quora-question-answer-dataset")
dataset = dataset["train"].train_test_split(test_size=0.2)
train_dataset = dataset["train"]
test_dataset = dataset["test"]



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/56402 [00:00<?, ? examples/s]

#**Data Pre-Processing**

In [6]:

def preprocess_text(text):
    tokens = word_tokenize(text)
    tokens = [token for token in tokens if token.lower() not in stop_words]
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    return ' '.join(tokens)

# Apply preprocessing to the 'question' and 'answer' columns
dataset = dataset.map(lambda x: {'question': preprocess_text(x['question']),
                                 'answer': preprocess_text(x['answer'])})

# Convert dataset to pandas DataFrame for splitting
df = dataset['train'].to_pandas()

# Split the data into training and validation sets
train_data, val_data = train_test_split(df, test_size=0.2, random_state=42)

print("Training Data:")
print(train_data)
print("\nValidation Data:")
print(val_data)


Map:   0%|          | 0/45121 [00:00<?, ? examples/s]

Map:   0%|          | 0/11281 [00:00<?, ? examples/s]

Training Data:
                                                question  \
17394                                          calculate   
29099                            imagine life 100 year ?   
42067                              represents GOP 2021 ?   
24706                   teenager wish adult understood ?   
35644                           ever judged appearance ?   
...                                                  ...   
11284           happen $ 1 equal 1 rupee ? consequence ?   
44732                  one find best biryani bangalore ?   
38158  British Empire able become big/powerful UK sma...   
860    modern English le Celtic word Franco-Greco-Lat...   
15795      find stock swing trading positional trading ?   

                                                  answer  
17394  sum first n natural number : ( n ) = ( n ( n+1...  
29099                                          Extinct ?  
42067                                   Mitch McCarthy .  
24706  • stupid . opinion va

#**Split the training data into train and validation sets**

In [8]:


# Split the training data into train and validation sets (80/20)
train_df, val_df = train_test_split(train_data, test_size=0.2, random_state=42)

print("Train DataFrame:")
print(train_df)
print("\nValidation DataFrame:")
print(val_df)


Train DataFrame:
                                                question  \
26131              use WhatsApp without SIM card phone ?   
31497  vocal exercise really help singing become bett...   
14142          lose 10 pound 3 week without exercising ?   
18243                   genetic malformation Megan Fox ?   
32148  purchase Marrow note prepare note second-year ...   
...                                                  ...   
17694                                   best thing die ?   
22272              homosexual decide alpha ; coin toss ?   
32187                                  see picture dog ?   
22864   cosmetic dentistry different general dentistry ?   
16047             without using mathematical induction ?   

                                                  answer  
26131  would like know tov get old watsapp without od...  
31497  Oh , absolutely , technically correct exercise...  
14142  60 Second Dental Trick Going Bed Tonight Rebui...  
18243  genetic malformatio

In [9]:

def preprocess_text(text):
    tokens = word_tokenize(text)
    tokens = [token for token in tokens if token.lower() not in stop_words]
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    return ' '.join(tokens)

# Apply preprocessing to the 'question' and 'answer' columns
dataset = dataset.map(lambda x: {'question': preprocess_text(x['question']),
                                 'answer': preprocess_text(x['answer'])})

# Convert dataset to pandas DataFrame for splitting
df = dataset['train'].to_pandas()

# Split the data into training and validation sets
train_data, val_data = train_test_split(df, test_size=0.2, random_state=42)

print("Training Data:")
print(train_data)
print("\nValidation Data:")
print(val_data)


Map:   0%|          | 0/45121 [00:00<?, ? examples/s]

Map:   0%|          | 0/11281 [00:00<?, ? examples/s]

Training Data:
                                                question  \
17394                                          calculate   
29099                            imagine life 100 year ?   
42067                              represents GOP 2021 ?   
24706                   teenager wish adult understood ?   
35644                           ever judged appearance ?   
...                                                  ...   
11284           happen $ 1 equal 1 rupee ? consequence ?   
44732                  one find best biryani bangalore ?   
38158  British Empire able become big/powerful UK sma...   
860    modern English le Celtic word Franco-Greco-Lat...   
15795      find stock swing trading positional trading ?   

                                                  answer  
17394  sum first n natural number : ( n ) = ( n ( n+1...  
29099                                          Extinct ?  
42067                                   Mitch McCarthy .  
24706  • stupid . opinion va

#Data Cleaning

Clean the text


In [10]:
import re

def clean_text(text):
  # Remove special characters and numbers
  text = re.sub(r'[^a-zA-Z\s]', '', text)
  # Convert to lowercase
  text = text.lower()
  # Remove extra whitespace
  text = text.strip()
  return text

# Apply clean_text function to 'question' and 'answer' columns in train_df
train_df['question'] = train_df['question'].apply(clean_text)
train_df['answer'] = train_df['answer'].apply(clean_text)

# Apply clean_text function to 'question' and 'answer' columns in val_df
val_df['question'] = val_df['question'].apply(clean_text)
val_df['answer'] = val_df['answer'].apply(clean_text)

print("Cleaned Train DataFrame:")
print(train_df.head())
print("\nCleaned Validation DataFrame:")
print(val_df.head())

Cleaned Train DataFrame:
                                                question  \
26131                use whatsapp without sim card phone   
31497   vocal exercise really help singing become better   
14142               lose  pound  week without exercising   
18243                     genetic malformation megan fox   
32148  purchase marrow note prepare note secondyear m...   

                                                  answer  
26131  would like know tov get old watsapp without od...  
31497  oh  absolutely  technically correct exercise  ...  
14142  second dental trick going bed tonight rebuild ...  
18243  genetic malformation called  stupidity   lucky...  
32148  nd mbbs student  ve recently purchased marrow ...  

Cleaned Validation DataFrame:
                                                question  \
36360               lpu good taking admission cse branch   
3569   english almost grammarless language   richard ...   
28659                 rohit shetty directed inter

# Lemmatization




In [12]:


def lemmatize_text(text):
  tokens = word_tokenize(text)
  lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]
  return ' '.join(lemmatized_tokens)

# Apply lemmatize_text function to 'question' and 'answer' columns in train_df
train_df['question'] = train_df['question'].apply(lemmatize_text)
train_df['answer'] = train_df['answer'].apply(lemmatize_text)

# Apply lemmatize_text function to 'question' and 'answer' columns in val_df
val_df['question'] = val_df['question'].apply(lemmatize_text)
val_df['answer'] = val_df['answer'].apply(lemmatize_text)

print("Lemmatized Train DataFrame:")
print(train_df.head())
print("\nLemmatized Validation DataFrame:")
print(val_df.head())

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Lemmatized Train DataFrame:
                                                question  \
26131                use whatsapp without sim card phone   
31497   vocal exercise really help singing become better   
14142                 lose pound week without exercising   
18243                     genetic malformation megan fox   
32148  purchase marrow note prepare note secondyear m...   

                                                  answer  
26131  would like know tov get old watsapp without od...  
31497  oh absolutely technically correct exercise aim...  
14142  second dental trick going bed tonight rebuild ...  
18243  genetic malformation called stupidity lucky la...  
32148  nd mbbs student ve recently purchased marrow l...  

Lemmatized Validation DataFrame:
                                                question  \
36360               lpu good taking admission cse branch   
3569   english almost grammarless language richard gr...   
28659                 rohit shetty directed

# **Data Augmentation**

 # Paraphrasing

In [14]:


!pip install nlpaug
import nlpaug.augmenter.word as naw

# Initialize a synonym augmenter
aug = naw.SynonymAug(aug_src='wordnet')

# Function to paraphrase text
def paraphrase_text(text):
  augmented_text = aug.augment(text)
  return augmented_text

# Apply paraphrasing to 'question' column in train_df
train_df['question'] = train_df['question'].apply(paraphrase_text)

print("Paraphrased Train DataFrame:")
print(train_df.head())


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.5/410.5 kB 7.5 MB/s eta 0:00:00


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


Paraphrased Train DataFrame:
                                            question               answer  \
0  [What is the capital of Jacques anatole franco...                Paris   
1                             [Who compose Village?]  William Shakespeare   

             paraphrased_question    paraphrased_answer  
0  What is the capital of France?                Paris.  
1               Who wrote Hamlet?  William Shakespeare.  


In [ ]:
# prompt: apply models further

# **BERT Model**

# Load pre-trained BERT model and tokenizer
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

# Tokenize and encode input data (example using the first question in train_df)
inputs = tokenizer(train_df['question'].iloc[0], return_tensors='pt')

# Forward pass through the model
outputs = model(**inputs)

# Extract embeddings (e.g., last hidden state)
embeddings = outputs.last_hidden_state

# Visualization (example: dimensionality reduction and plotting)
from sklearn.manifold import TSNE
tsne = TSNE(n_components=2)
embeddings_2d = tsne.fit_transform(embeddings.detach().numpy().squeeze())

plt.scatter(embeddings_2d[:, 0], embeddings_2d[:, 1])
plt.title('BERT Embeddings Visualization')
plt.show()

# **GPT Model**

# Load pre-trained GPT model and tokenizer
model_name = 'gpt2'
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2Model.from_pretrained(model_name)

# Tokenize and encode input data (example using the first answer in train_df)
inputs = tokenizer(train_df['answer'].iloc[0], return_tensors='pt')

# Forward pass through the model
outputs = model(**inputs)

# Extract embeddings (e.g., last hidden state)
embeddings = outputs.last_hidden_state

# Visualization (example: dimensionality reduction and plotting)
tsne = TSNE(n_components=2)
embeddings_2d = tsne.fit_transform(embeddings.detach().numpy().squeeze())

plt.scatter(embeddings_2d[:, 0], embeddings_2d[:, 1])
plt.title('GPT Embeddings Visualization')
plt.show()

# **T5 Model**

# Load pre-trained T5 model and tokenizer
model_name = 't5-small'
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5Model.from_pretrained(model_name)

# Tokenize and encode input data (example using the first question-answer pair in train_df)
inputs = tokenizer(train_df['question'].iloc[0], train_df['answer'].iloc[0], return_tensors='pt')

# Forward pass through the model
outputs = model(**inputs)

# Extract embeddings (e.g., last hidden state)
embeddings = outputs.last_hidden_state

# Visualization (example: heatmap of attention weights)
plt.imshow(outputs.encoder_last_hidden_state[0, :, :].detach().numpy())
plt.title('T5 Attention Weights Visualization')
plt.colorbar()
plt.show()


In [21]:
# Printing the organized data
for i, key in enumerate(data, start=1):
    print(f"Entry {i}")
    print(f"Question: {data[key][i-1]}") # Access the list element using an integer index (i-1)
    print(f"Answer: {data[key][i-1]}")   # Access the list element using an integer index (i-1)
    print()

Entry 1
Question: ?
Answer: ?

Entry 2
Question: I am fine, thank you.
Answer: I am fine, thank you.



In [ ]:
# prompt: Handling Contractions: Expand contractions (e.g., "don't" to "do not") to standardize the text.

!pip install contractions
import contractions

def expand_contractions(text):
  expanded_text = contractions.fix(text)
  return expanded_text

df['question_expanded'] = df['question_no_punc'].apply(expand_contractions)
df['answer_expanded'] = df['answer_no_punc'].apply(expand_contractions)

print(df[['question_expanded', 'answer_expanded']])


In [ ]:
# prompt:  Removing URLs, Digits and White Spaces

# Remove URLs
df['question_no_url'] = df['question_no_punc'].apply(lambda x: re.sub(r'http\S+', '', x))
df['answer_no_url'] = df['answer_no_punc'].apply(lambda x: re.sub(r'http\S+', '', x))

# Remove Digits
df['question_no_digits'] = df['question_no_url'].apply(lambda x: re.sub(r'\d+', '', x))
df['answer_no_digits'] = df['answer_no_url'].apply(lambda x: re.sub(r'\d+', '', x))

# Remove Extra White Spaces
df['question_no_spaces'] = df['question_no_digits'].apply(lambda x: ' '.join(x.split()))
df['answer_no_spaces'] = df['answer_no_digits'].apply(lambda x: ' '.join(x.split()))

print(df[['question_no_spaces', 'answer_no_spaces']])


In [ ]:
# prompt: Check if  Removing URLs, Digits and White Spaces are eliminated

# Check for URLs
print("URLs in questions:", df['question_no_spaces'].str.contains('http').any())
print("URLs in answers:", df['answer_no_spaces'].str.contains('http').any())

# Check for digits
print("Digits in questions:", df['question_no_spaces'].str.contains('\d').any())
print("Digits in answers:", df['answer_no_spaces'].str.contains('\d').any())

# Check for extra white spaces (more than one space in a row)
print("Extra spaces in questions:", df['question_no_spaces'].str.contains('\s{2,}').any())
print("Extra spaces in answers:", df['answer_no_spaces'].str.contains('\s{2,}').any())


In [ ]:
# prompt: Clean the text using NLP techniques to remove urls , stopwords and special characters

nltk.download('punkt')
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

def clean_text(text):
    # Remove URLs
    text = re.sub(r"http\S+", "", text)

    # Remove special characters and keep only alphanumeric and spaces
    text = re.sub(r"[^a-zA-Z0-9\s]", "", text)

    # Tokenize the text
    tokens = word_tokenize(text.lower())

    # Remove stop words
    filtered_tokens = [word for word in tokens if word not in stop_words]

    # Join the tokens back into a string
    cleaned_text = " ".join(filtered_tokens)

    return cleaned_text

# Apply the clean_text function to the 'question' and 'answer' columns of your DataFrame
df['question_cleaned'] = df['question'].apply(clean_text)
df['answer_cleaned'] = df['answer'].apply(clean_text)

print(df[['question_cleaned', 'answer_cleaned']])
